In [12]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd

import re
import spacy

from textblob import TextBlob

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from scipy.stats import pearsonr
from scipy.stats import ttest_ind

import transformers
from transformers import AutoTokenizer, BertModel

### Rows and columns with maximum number of non-zero values

In [28]:
table = np.array([
    [ 'Country', 'Population', 'Area (km²)', 'GDP per capita (USD)'],
    [ 'USA', '331,449,281', '9,826,630', '62,794'],
    [ 'Canada', '38,048,738', '9,984,670', '48,265'],
    [ 'Mexico', '129,166,028', '1,964,375', '9,747'],
    [ 'Argentina', '45,197,956', '2,780,400', '11,866'],
    [ 'Brazil', '212,559,417', '8,515,767', '9,126']
])

non_zero_counts = np.count_nonzero(table[1:], axis=1)
max_non_zero_count = np.max(non_zero_counts)
max_non_zero_rows = table[1:][non_zero_counts == max_non_zero_count]

print("Rows with maximum number of non-zero values:")
for row in max_non_zero_rows:
    print(row)

non_zero_counts = np.count_nonzero(table[1:], axis=0)
max_non_zero_count = np.max(non_zero_counts)
max_non_zero_cols = table[:,non_zero_counts == max_non_zero_count]

print("Columns with maximum number of non-zero values:")
for col in max_non_zero_cols.T:
    print(col)

Rows with maximum number of non-zero values:
['USA' '331,449,281' '9,826,630' '62,794']
['Canada' '38,048,738' '9,984,670' '48,265']
['Mexico' '129,166,028' '1,964,375' '9,747']
['Argentina' '45,197,956' '2,780,400' '11,866']
['Brazil' '212,559,417' '8,515,767' '9,126']
Columns with maximum number of non-zero values:
['Country' 'USA' 'Canada' 'Mexico' 'Argentina' 'Brazil']
['Population' '331,449,281' '38,048,738' '129,166,028' '45,197,956'
 '212,559,417']
['Area (km²)' '9,826,630' '9,984,670' '1,964,375' '2,780,400' '8,515,767']
['GDP per capita (USD)' '62,794' '48,265' '9,747' '11,866' '9,126']


### Rows with the most frequent values as relevant

In [29]:
data = {
    "Country": ["USA", "Canada", "Mexico", "Argentina", "Brazil"],
    "Population": [331449281, 38048738, 129166028, 45197956, 212559417],
    "Area (km²)": [9826630, 9984670, 1964375, 2780400, 8515767],
    "GDP per capita (USD)": [62794, 48265, 9747, 11866, 9126]
}
table = pd.DataFrame(data)

most_frequent_row = table.mode().iloc[0]

relevant_rows = table[table.eq(most_frequent_row).any(axis=1)]

print(relevant_rows)

     Country  Population  Area (km²)  GDP per capita (USD)
1     Canada    38048738     9984670                 48265
2     Mexico   129166028     1964375                  9747
3  Argentina    45197956     2780400                 11866
4     Brazil   212559417     8515767                  9126


### Columns with the highest standard deviation as relevant

In [30]:
data = {
    "Country": ["USA", "Canada", "Mexico", "Argentina", "Brazil"],
    "Population": [331449281, 38048738, 129166028, 45197956, 212559417],
    "Area (km²)": [9826630, 9984670, 1964375, 2780400, 8515767],
    "GDP per capita (USD)": [62794, 48265, 9747, 11866, 9126]
}
table = pd.DataFrame(data)

highest_std_col = table.std().idxmax()

relevant_cols = table.loc[:, [highest_std_col]]

print(relevant_cols)

   Population
0   331449281
1    38048738
2   129166028
3    45197956
4   212559417


/tmp/ipykernel_371043/2781234655.py:9: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  highest_std_col = table.std().idxmax()


### Columns with a high percentage of non-empty cells

In [31]:
data = {
    "Country": ["USA", "Canada", "Mexico", "Argentina", "Brazil"],
    "Population": [331449281, 38048738, 129166028, 45197956, 212559417],
    "Area (km²)": [9826630, 9984670, 1964375, 2780400, 8515767],
    "GDP per capita (USD)": [62794, 48265, 9747, 11866, 9126]
}
table = pd.DataFrame(data)

non_empty_percent = table.count() / table.shape[0] * 100

relevant_cols = table.loc[:, non_empty_percent > 50]

print(relevant_cols)

     Country  Population  Area (km²)  GDP per capita (USD)
0        USA   331449281     9826630                 62794
1     Canada    38048738     9984670                 48265
2     Mexico   129166028     1964375                  9747
3  Argentina    45197956     2780400                 11866
4     Brazil   212559417     8515767                  9126


### Rows that contain aggregate data such as sums, averages, etc. can also be considered important

In [34]:
data = {'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
        'Population': [8336817, 3979576, 2693976, 2320268, 1680992],
        'Median Age': [36.4, 35.2, 34.1, 33.8, 33.6],
        'Average Income (USD)': [77000, 65000, 63000, 59000, 55000],
        'Average Rent (USD)': [2900, 2300, 1800, 1200, 1100]}

df = pd.DataFrame(data)

agg_cols = ['Population', 'Median Age', 'Average Income (USD)', 'Average Rent (USD)']

df['Mean Population'] = df[agg_cols].mean(axis=1)
df['Sum Avg Income and Rent'] = df['Average Income (USD)'] + df['Average Rent (USD)']

sorted_df = df.sort_values(by=['Mean Population', 'Sum Avg Income and Rent'], ascending=[False, False])

print(sorted_df)

          City  Population  Median Age  Average Income (USD)  \
0     New York     8336817        36.4                 77000   
1  Los Angeles     3979576        35.2                 65000   
2      Chicago     2693976        34.1                 63000   
3      Houston     2320268        33.8                 59000   
4      Phoenix     1680992        33.6                 55000   

   Average Rent (USD)  Mean Population  Sum Avg Income and Rent  
0                2900      2104188.350                    79900  
1                2300      1011727.800                    67300  
2                1800       689702.525                    64800  
3                1200       595125.450                    60200  
4                1100       434281.400                    56100  


### Correlation: Relevant rows and columns can be selected based on the correlation of the data in a column with the data in other columns.

| City      | Population | Area (km²) | Average temperature (°C) | Elevation (m) |
|-----------|------------|------------|--------------------------|---------------|
| Tokyo     | 13,515,271 | 2,187      | 16.2                     | 44            |
| New York  | 8,175,133  | 468.9      | 12.9                     | 10            |
| London    | 8,982,000  | 1,572      | 9.8                      | 35            |
| Paris     | 2,148,271  | 105.4      | 11.5                     | 35            |
| Istanbul  | 15,029,231 | 5,461      | 13.9                     | 110           |


In [35]:
data = {'City': ['Tokyo', 'New York', 'London', 'Paris', 'Istanbul'],
        'Population': [13515271, 8175133, 8982000, 2148271, 15029231],
        'Area (km²)': [2187, 468.9, 1572, 105.4, 5461],
        'Average temperature (°C)': [16.2, 12.9, 9.8, 11.5, 13.9],
        'Elevation (m)': [44, 10, 35, 35, 110]}
df = pd.DataFrame(data)

corr_matrix = df.corr()

relevant_columns = []
threshold = 0.8
for col in corr_matrix.columns:
    if max(corr_matrix[col]) > threshold:
        relevant_columns.append(col)

df_relevant = df[relevant_columns]

print(df_relevant)

   Population  Area (km²)  Average temperature (°C)  Elevation (m)
0    13515271      2187.0                      16.2             44
1     8175133       468.9                      12.9             10
2     8982000      1572.0                       9.8             35
3     2148271       105.4                      11.5             35
4    15029231      5461.0                      13.9            110


/tmp/ipykernel_371043/3838197159.py:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


### Specific pattern match

In [4]:
table = [
    ["Name", "Age", "City"],
    ["John Smith", "32", "New York City"],
    ["Jane Doe", "25", "Los Angeles"],
    ["Bob Johnson", "43", "Chicago"],
    ["Alice Williams", "29", "San Francisco"]
]

city_regex = re.compile(r"^(New York City|Los Angeles|Chicago)$")

city_col_index = None
for i, col in enumerate(table[0]):
    if col == "City":
        city_col_index = i
        break

selected_rows = []
for row in table[1:]:
    if city_regex.match(row[city_col_index]):
        selected_rows.append(row)

print(selected_rows)

[['John Smith', '32', 'New York City'], ['Jane Doe', '25', 'Los Angeles'], ['Bob Johnson', '43', 'Chicago']]


### NER

In [2]:
nlp = spacy.load("en_core_web_sm")

table_data = [
    ["Product", "Description", "Price"],
    ["Apples", "Fresh, juicy apples from our orchard", "$0.99"],
    ["Bananas", "Ripe, yellow bananas from Ecuador", "$0.69"],
    ["Oranges", "Sweet and tangy Florida oranges", "$1.25"],
    ["Grapes", "Plump grapes from California vineyards", "$1.49"],
    ["Strawberries", "Fresh strawberries from local farms", "$2.99"],
    ["Blueberries", "Juicy blueberries from Maine", "$3.99"],
    ["Raspberries", "Sweet and tart raspberries from Oregon", "$4.99"],
    ["Peaches", "Juicy, ripe peaches from Georgia", "$1.99"],
]

relevant_rows = []
relevant_cols = []

for row in table_data:
    relevant_flag = False
    
    for cell in row:
        doc = nlp(cell)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        
        if any(ent[1] == "GPE" for ent in entities):
            relevant_flag = True
            break
        
        blob = TextBlob(cell)
        sentiment = blob.sentiment.polarity
        
        if sentiment > 0:
            relevant_flag = True
            break
    
    if relevant_flag:
        relevant_rows.append(row)
    
    for i, cell in enumerate(row):
        doc = nlp(cell)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        
        if any(ent[1] == "GPE" for ent in entities):
            relevant_cols.append(i)
            break
        
        blob = TextBlob(cell)
        sentiment = blob.sentiment.polarity
        
        if sentiment > 0:
            relevant_cols.append(i)
            break

print("Relevant Rows:")
for row in relevant_rows:
    print(row)

print("\nRelevant Columns:")
for col in relevant_cols:
    column_data = [row[col] for row in table_data]
    print(column_data)

Relevant Rows:
['Apples', 'Fresh, juicy apples from our orchard', '$0.99']
['Bananas', 'Ripe, yellow bananas from Ecuador', '$0.69']
['Oranges', 'Sweet and tangy Florida oranges', '$1.25']
['Grapes', 'Plump grapes from California vineyards', '$1.49']
['Strawberries', 'Fresh strawberries from local farms', '$2.99']
['Blueberries', 'Juicy blueberries from Maine', '$3.99']
['Raspberries', 'Sweet and tart raspberries from Oregon', '$4.99']
['Peaches', 'Juicy, ripe peaches from Georgia', '$1.99']

Relevant Columns:
['Description', 'Fresh, juicy apples from our orchard', 'Ripe, yellow bananas from Ecuador', 'Sweet and tangy Florida oranges', 'Plump grapes from California vineyards', 'Fresh strawberries from local farms', 'Juicy blueberries from Maine', 'Sweet and tart raspberries from Oregon', 'Juicy, ripe peaches from Georgia']
['Product', 'Apples', 'Bananas', 'Oranges', 'Grapes', 'Strawberries', 'Blueberries', 'Raspberries', 'Peaches']
['Description', 'Fresh, juicy apples from our orchard'

### very basic and similar to https://arxiv.org/pdf/2005.09207.pdf

In [17]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
def encode_table_query(model, table, query):
    tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
    inputs = tokenizer.encode_plus(table, query, add_special_tokens=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [19]:
def retrieve_relevant_info(table_representation, query_representation):
    scores = torch.nn.functional.cosine_similarity(table_representation, query_representation)
    relevant_rows = scores.topk(k=1, dim=0).indices.tolist()
    return relevant_rows

In [20]:
table = "Name | Capital | Population\n------------------------------\nFrance| Paris  | 66 million\n------------------------------\nGermany| Berlin | 83 million\n------------------------------\nItaly | Rome   | 60 million"
query = "Capital and population of Cities?"

table_representation = encode_table_query(model, table, "")
query_representation = encode_table_query(model, "", query)
relevant_rows = retrieve_relevant_info(table_representation, query_representation)

print("Relevant rows:", relevant_rows)

Relevant rows: [0]


In [21]:
def encode_table(model, table, caption, reference):
    input_text = caption + " " + reference + " " + table
    input_ids = transformers.BertTokenizer.from_pretrained('bert-base-uncased').encode(input_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs[0].mean(dim=1)

In [22]:
class RelevancePredictor(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(RelevancePredictor, self).__init__()
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc(x)
        return x

In [23]:
table = "Name | Capital | Population\n------------------------------\nFrance| Paris  | 66 million\n------------------------------\nGermany| Berlin | 83 million\n------------------------------\nItaly | Rome   | 60 million"
caption = "Information about countries"
reference = "Source: Wikipedia"

table_representation = encode_table(model, table, caption, reference)
model = RelevancePredictor(hidden_size=table_representation.size(-1), num_classes=1)
predictions = model(table_representation)

print("Relevance predictions:", predictions)

Relevance predictions: tensor([[-0.2970]], grad_fn=<AddmmBackward0>)


### Basic Idea

| Fruit   | Color  | Price |
|---------|--------|-------|
| Apple   | Red    | 0.99  |
| Orange  | Orange | 1.25  |
| Banana  | Yellow | 0.69  |
| Kiwi    | Brown  | 1.50  |


[CLS] Fruit [SEP] Color [SEP] Price [SEP]
Apple [SEP] Red [SEP] 0.99 [SEP]
Orange [SEP] Orange [SEP] 1.25 [SEP]
Banana [SEP] Yellow [SEP] 0.69 [SEP]
Kiwi [SEP] Brown [SEP] 1.50 [SEP]
[SEP]

### Encoding table information ... Similar to TaBERT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
table_data = [
    ["Name", "Age", "Gender"],
    ["Alice", "25", "Female"],
    ["Bob", "30", "Male"],
    ["Carol", "40", "Female"]
]

In [12]:
cells = []

for i, row in enumerate(table):
    for j, cell in enumerate(row):
        cell_tokens = tokenizer.encode(cell, add_special_tokens=False)
        position_tokens = tokenizer.encode(f'[{i}][{j}]')
        cells.append(cell_tokens + position_tokens)

max_len = max(len(cell) for cell in cells)
padded_cells = [cell + [tokenizer.pad_token_id] * (max_len - len(cell)) for cell in cells]
input_ids = torch.tensor(padded_cells)

print(input_ids)

tensor([[2171,  101, 1031, 1014, 1033, 1031, 1014, 1033,  102],
        [2287,  101, 1031, 1014, 1033, 1031, 1015, 1033,  102],
        [5907,  101, 1031, 1014, 1033, 1031, 1016, 1033,  102],
        [2198,  101, 1031, 1015, 1033, 1031, 1014, 1033,  102],
        [2423,  101, 1031, 1015, 1033, 1031, 1015, 1033,  102],
        [3287,  101, 1031, 1015, 1033, 1031, 1016, 1033,  102],
        [4532,  101, 1031, 1016, 1033, 1031, 1014, 1033,  102],
        [3590,  101, 1031, 1016, 1033, 1031, 1015, 1033,  102],
        [2931,  101, 1031, 1016, 1033, 1031, 1016, 1033,  102],
        [3419,  101, 1031, 1017, 1033, 1031, 1014, 1033,  102],
        [2539,  101, 1031, 1017, 1033, 1031, 1015, 1033,  102],
        [3287,  101, 1031, 1017, 1033, 1031, 1016, 1033,  102]])


In [15]:
cells = []
for i, row in enumerate(table):
    for j, cell in enumerate(row):
        cell_tokens = tokenizer.encode(cell, add_special_tokens=False)
        position_tokens = tokenizer.encode(f'[{i+1}] [{j}]')
        if i == 0:
            header_tokens = tokenizer.encode(cell, add_special_tokens=False)
            cells.append(header_tokens + [tokenizer.sep_token_id] + cell_tokens + position_tokens)
        else:
            cells.append(cell_tokens + position_tokens)

max_len = max(len(cell) for cell in cells)
padded_cells = [cell + [tokenizer.pad_token_id] * (max_len - len(cell)) for cell in cells]
input_ids = torch.tensor(padded_cells)

print(input_ids)

tensor([[2171,  102, 2171,  101, 1031, 1015, 1033, 1031, 1014, 1033,  102],
        [2287,  102, 2287,  101, 1031, 1015, 1033, 1031, 1015, 1033,  102],
        [5907,  102, 5907,  101, 1031, 1015, 1033, 1031, 1016, 1033,  102],
        [2198,  101, 1031, 1016, 1033, 1031, 1014, 1033,  102,    0,    0],
        [2423,  101, 1031, 1016, 1033, 1031, 1015, 1033,  102,    0,    0],
        [3287,  101, 1031, 1016, 1033, 1031, 1016, 1033,  102,    0,    0],
        [4532,  101, 1031, 1017, 1033, 1031, 1014, 1033,  102,    0,    0],
        [3590,  101, 1031, 1017, 1033, 1031, 1015, 1033,  102,    0,    0],
        [2931,  101, 1031, 1017, 1033, 1031, 1016, 1033,  102,    0,    0],
        [3419,  101, 1031, 1018, 1033, 1031, 1014, 1033,  102,    0,    0],
        [2539,  101, 1031, 1018, 1033, 1031, 1015, 1033,  102,    0,    0],
        [3287,  101, 1031, 1018, 1033, 1031, 1016, 1033,  102,    0,    0]])


### Using statistical measures

In [10]:
# In this example, I am selecting the relevant cells based on their correlation with GDP.
# In this example, I have selected the relevant cells (Population and GDP) based on their correlation with GDP.

data = pd.DataFrame({
    'Country': ['USA', 'China', 'India', 'Russia', 'Japan'],
    'Population': [331, 1393, 1380, 144, 126],
    'GDP': [21428, 14732, 2955, 1678, 5068],
    'Life Expectancy': [78.9, 76.9, 69.7, 72.3, 84.6]
})
data = data.set_index('Country')

# select relevant cells based on correlation with GDP
relevant_columns = ['Population', 'GDP']
correlations = data[relevant_columns].apply(lambda x: pearsonr(x, data['GDP']))
correlations_df = pd.DataFrame({'Correlation': [corr[0] for corr in correlations], 'P-Value': [corr[1] for corr in correlations]}, index=relevant_columns)
relevant_data = data[relevant_columns]

print(relevant_data.join(correlations_df))

         Population    GDP Correlation P-Value
Country                                       
USA             331  21428         NaN     NaN
China          1393  14732         NaN     NaN
India          1380   2955         NaN     NaN
Russia          144   1678         NaN     NaN
Japan           126   5068         NaN     NaN


In [ ]:
# In this example, I am selecting the relevant cells based on k-means clustering and linear regression. 
# to cluster the data into two clusters based on the Population and GDP columns. 
# I have then selected the cells from the second cluster as they have a higher GDP. 
# I have also used LinearRegression to fit a linear regression model to predict the GDP based on the population.

# select relevant cells based on k-means clustering
relevant_columns = ['Population', 'GDP']
kmeans = KMeans(n_clusters=2)
clusters = kmeans.fit_predict(data[relevant_columns])
relevant_data = data[clusters == 1]

# select relevant cells based on linear regression
regression = LinearRegression()
X = data['Population'].values.reshape(-1, 1)
y = data['GDP'].values.reshape(-1, 1)
regression.fit(X, y)
predicted_GDP = regression.predict(X)
relevant_data['Predicted GDP'] = predicted_GDP
relevant_columns.append('Predicted GDP')

print(relevant_data[relevant_columns])

In [14]:
# In this example, I am selecting the relevant cells based on a t-test to determine if there is a significant difference between each country's data and the mean of all countries in the columns.
# I have performed the t-test on each column, and have selected the columns with a p-value less than 0.05.

# perform a t-test to determine the significant columns
relevant_columns = []
for column in data.columns:
    country_data = data[column]
    t_statistic, p_value = ttest_ind(country_data, data.mean())
    if p_value < 0.05:
        relevant_columns.append(column)

print(data[relevant_columns])

Empty DataFrame
Columns: []
Index: [USA, China, India, Russia, Japan]


### Using unsupervised methods

### Using clustering

In [40]:
table_data = [
    ['City', 'Population', 'Area (km²)', 'GDP per capita (USD)', 'Elevation (meters)'],
    ['New York City', '8,336,817', '468.9', '93,574', '10'],
    ['Los Angeles', '3,979,576', '1,213.9', '62,820', '71'],
    ['Chicago', '2,693,976', '606.1', '69,188', '182'],
    ['Houston', '2,320,268', '1,651.1', '62,095', '18'],
    ['Phoenix', '1,680,992', '1,338.3', '54,956', '331'],
    ['Philadelphia', '1,584,064', '347.6', '60,116', '12'],
    ['San Antonio', '1,547,253', '1,067.3', '47,419', '198'],
    ['San Diego', '1,423,851', '372.4', '64,334', '22'],
    ['Dallas', '1,343,573', '997.0', '61,347', '137'],
    ['San Jose', '1,015,785', '467.2', '82,880', '27']
]

num_data = []
for row in table_data[1:]:
    num_row = []
    for val in row[1:]:
        val = val.replace(',', '')  # remove commas in numbers
        try:
            num_row.append(float(val))
        except ValueError:
            num_row.append(np.nan)
    num_data.append(num_row)

kmeans = KMeans(n_clusters=2).fit(num_data)
labels = kmeans.labels_

for i, row in enumerate(table_data[1:]):
    print(f"{row[0]}: Cluster {labels[i]}")

New York City: Cluster 1
Los Angeles: Cluster 0
Chicago: Cluster 0
Houston: Cluster 0
Phoenix: Cluster 0
Philadelphia: Cluster 0
San Antonio: Cluster 0
San Diego: Cluster 0
Dallas: Cluster 0
San Jose: Cluster 0


### Using PCA

In [3]:
data = pd.DataFrame({
    'Country': ['USA', 'China', 'India', 'Russia', 'Japan'],
    'Population': [331, 1393, 1380, 144, 126],
    'GDP': [21428, 14732, 2955, 1678, 5068],
    'Life Expectancy': [78.9, 76.9, 69.7, 72.3, 84.6]
})

data = data.set_index('Country')

features = data.values

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)

selected_cells = pd.DataFrame(principal_components, index=data.index, columns=['PC1', 'PC2'])

print(selected_cells.head())

              PC1       PC2
Country                    
USA     -1.284715 -0.968521
China    0.365595 -1.310261
India    1.944905 -0.001206
Russia   0.438567  1.353606
Japan   -1.464353  0.926382


In [4]:
# I have added some domain knowledge to interpret the results of the PCA.

# Population Density: This column is calculated by dividing the first principal component (PC1) by the population of each country. 
# A measure of the population density of each country.

# Development Index: This column is calculated by multiplying the second principal component (PC2) by the GDP of each country and dividing by the life expectancy. 
# A measure of the development index of each country.

# add some domain knowledge to interpret the results
selected_cells['Population Density'] = selected_cells['PC1'] / data['Population']
selected_cells['Development Index'] = selected_cells['PC2'] * data['GDP'] / data['Life Expectancy']

print(selected_cells.head())

              PC1       PC2  Population Density  Development Index
Country                                                           
USA     -1.284715 -0.968521           -0.003881        -263.035035
China    0.365595 -1.310261            0.000262        -251.011329
India    1.944905 -0.001206            0.001409          -0.051114
Russia   0.438567  1.353606            0.003046          31.415639
Japan   -1.464353  0.926382           -0.011622          55.495316


In [6]:
# In this example, I have added three additional techniques to provide insights into the relationships between the cells in the table:

# Clustering: I have used k-means clustering to group the countries into two clusters based on their principal components. 
# I have added a new column to the resulting table (Cluster) to indicate which cluster each country belongs to.

# Regression: I have used linear regression to fit the first principal component (PC1) to the population of each country.

# Hypothesis testing: We have used Pearson's correlation coefficient to test the relationship between the GDP and the second principal component (PC2). 
# Two new columns to the resulting table (GDP Correlation and GDP P-Value) to show the strength and statistical significance of this relationship.

# add clustering information
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(selected_cells[['PC1', 'PC2']])
selected_cells['Cluster'] = clusters

# add regression information
reg = LinearRegression().fit(data[['Population']], selected_cells['PC1'])
selected_cells['PC1 Residuals'] = selected_cells['PC1'] - reg.predict(data[['Population']])
r, p = pearsonr(data['GDP'], selected_cells['PC2'])
selected_cells['GDP Correlation'] = r
selected_cells['GDP P-Value'] = p

print(selected_cells.head())

              PC1       PC2  Population Density  Development Index  Cluster  \
Country                                                                       
USA     -1.284715 -0.968521           -0.003881        -263.035035        1   
China    0.365595 -1.310261            0.000262        -251.011329        0   
India    1.944905 -0.001206            0.001409          -0.051114        0   
Russia   0.438567  1.353606            0.003046          31.415639        0   
Japan   -1.464353  0.926382           -0.011622          55.495316        1   

         PC1 Residuals  GDP Correlation  GDP P-Value  
Country                                               
USA          -0.753917        -0.836548     0.077353  
China        -0.743243        -0.836548     0.077353  
India         0.856138        -0.836548     0.077353  
Russia        1.258076        -0.836548     0.077353  
Japan        -0.617053        -0.836548     0.077353  


/lnet/work/people/mukherjee/own_code/venv_pytrch/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
